**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Oct  4 13:13:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 휴먼기계바이오공학부_노효영_1770025_과제3.ipynb
'Colab Notebooks'
'cough sound.zip (Unzipped Files)'
 data_spiral.csv
 DeepLearning-examples-master
 div2k_100
 이화인인증.JPG
'My Drive'
 PatternProject
 shallowNN_reference.ipynb
 SRCNN.ipynb
 Test_BoneClassification_reference.ipynb
'Train_BoneClassification_reference(1).ipynb'
'Train_BoneClassification_reference(2).ipynb'
 Train_BoneClassification_reference.ipynb
'train.csv.zip (Unzipped Files)'
 Untitled0.ipynb
 Untitled1.ipynb
'Untitled document.gdoc'
 yolov3


**1) Clone, configure & compile Darknet**



In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14321, done.
remote: Total 14321 (delta 0), reused 0 (delta 0), pack-reused 14321
Receiving objects: 100% (14321/14321), 12.86 MiB | 4.63 MiB/s, done.
Resolving deltas: 100% (9772/9772), done.


In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

**2) Configure yolov3.cfg file**

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=21@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=21@' cfg/yolov3_training.cfg

**3) Create .names and .data files**

In [ ]:
!echo -e 'Not Wearing Mask\nWearing Mask' > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

**4) Save yolov3_training.cfg and obj.names files in Google drive**

In [ ]:
!cp cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /mydrive/yolov3/classes.txt

**5) Create a folder and unzip image dataset**

In [ ]:
!mkdir data/obj1
!unzip /mydrive/yolov3/images.zip -d data/obj1

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj1/_104224787_tv050424063.jpg  
  inflating: data/obj1/_104224787_tv050424063.txt  
  inflating: data/obj1/_110755695_gettyimages-1202072951.jpg  
  inflating: data/obj1/_110755695_gettyimages-1202072951.txt  
  inflating: data/obj1/_111552161_mediaitem111552159.jpg  
  inflating: data/obj1/_111552161_mediaitem111552159.txt  
  inflating: data/obj1/_112342926_airportfacemask.jpg  
  inflating: data/obj1/_112342926_airportfacemask.txt  
  inflating: data/obj1/_DSC6585-1.jpg  
  inflating: data/obj1/_DSC6585-1.txt  
  inflating: data/obj1/004fJPG.jpg   
  inflating: data/obj1/004fJPG.txt   
  inflating: data/obj1/090518-people-sleeping-in-street.jpg  
  inflating: data/obj1/090518-people-sleeping-in-street.txt  
  inflating: data/obj1/10015042_701695786543261_283403204_o2028229_0_0.jpg  
  inflating: data/obj1/10015042_701695786543261_283403204_o2028229_0_0.txt  
  inflating: data/obj1/106470396-1585760884847gettyimages-1208880165.

**6) Create train.txt file**

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

**7) Download pre-trained weights for the convolutional layers file**

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-10-04 13:18:38--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   439KB/s    in 5m 51s  

2020-10-04 13:24:32 (452 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**8) Start training**

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

 3926: 0.120544, 0.090385 avg loss, 0.001000 rate, 11.968172 seconds, 251264 images, 0.355194 hours left
Loaded: 0.000062 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.845401, GIOU: 0.841832), Class: 0.999862, Obj: 0.999941, No Obj: 0.000232, .5R: 1.000000, .75R: 1.000000, count: 1, class_loss = 0.000000, iou_loss = 0.012034, total_loss = 0.012034 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.884748, GIOU: 0.881948), Class: 0.999665, Obj: 0.999791, No Obj: 0.000479, .5R: 1.000000, .75R: 1.000000, count: 5, class_loss = 0.000001, iou_loss = 0.052107, total_loss = 0.052108 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.868786, GIOU: 0.862515), Class: 0.999856, Obj: 0.999320, No Obj: 0.000058, .5R: 1.000000, .75R: 0.750000, count: 4, class_loss = 0.000003, iou_loss = 0.053939, total_loss = 0.053942 
 total_bbox = 529260, rewritten_bbox = 0.000000 % 
v3 (mse loss, Norma